In [3]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
import os

# print(tf.__version__)

1.13.1


In [ ]:
# Read data
# Folder paths
data_path = "/content/drive/My Drive/data_extract/"
out_path = "/content/drive/My Drive/Q2/prep_data"

x_train = np.load(data_path + "x_train.npy")
print(x_train.shape)
x_test = np.load(data_path + "x_test.npy")
print(x_test.shape)
# y_train = np.load(data_path + "y_train.npy")
# print(y_train.shape)
# y_test = np.load(data_path + "y_test.npy")
# print(y_test.shape)

In [2]:
tf.random.set_seed(1)
batch_size = 128
epochs = 10
learning_rate = 1e-3
momentum = 9e-1
intermediate_dim = 64
original_dim = 784

AttributeError: module 'tensorflow._api.v1.random' has no attribute 'set_seed'

In [ ]:
(training_features, _), _ = tf.keras.datasets.mnist.load_data()
training_features = training_features / np.max(training_features)
training_features = training_features.reshape(training_features.shape[0],
                                              training_features.shape[1] * training_features.shape[2]).astype(np.float32)
training_dataset = tf.data.Dataset.from_tensor_slices(training_features).batch(batch_size)

In [ ]:
class Encoder(tf.keras.layers.Layer):
  def __init__(self, intermediate_dim):
    super(Encoder, self).__init__()
    self.hidden_layer = tf.keras.layers.Dense(units=intermediate_dim, activation=tf.nn.relu)
    self.output_layer = tf.keras.layers.Dense(units=intermediate_dim, activation=tf.nn.relu)
    
  def call(self, input_features):
    activation = self.hidden_layer(input_features)
    return self.output_layer(activation)

In [ ]:
class Decoder(tf.keras.layers.Layer):
  def __init__(self, intermediate_dim, original_dim):
    super(Decoder, self).__init__()
    self.hidden_layer = tf.keras.layers.Dense(units=intermediate_dim, activation=tf.nn.relu)
    self.output_layer = tf.keras.layers.Dense(units=original_dim, activation=tf.nn.relu)
  
  def call(self, code):
    activation = self.hidden_layer(code)
    return self.output_layer(activation)

In [ ]:
class Autoencoder(tf.keras.Model):
  def __init__(self, intermediate_dim, original_dim):
    super(Autoencoder, self).__init__()
    self.encoder = Encoder(intermediate_dim=intermediate_dim)
    self.decoder = Decoder(intermediate_dim=intermediate_dim, original_dim=original_dim)
  
  def call(self, input_features):
    code = self.encoder(input_features)
    reconstructed = self.decoder(code)
    return reconstructed

In [ ]:
autoencoder = Autoencoder(intermediate_dim=intermediate_dim, original_dim=original_dim)
opt = tf.optimizers.SGD(learning_rate=learning_rate, momentum=momentum)

In [ ]:
def loss(model, original):
  reconstruction_error = tf.reduce_mean(tf.square(tf.subtract(model(original), original)))
  return reconstruction_error

In [ ]:
def train(loss, model, opt, original):
  with tf.GradientTape() as tape:
    gradients = tape.gradient(loss(model, original), model.trainable_variables)
    gradient_variables = zip(gradients, model.trainable_variables)
    opt.apply_gradients(gradient_variables)